In [20]:
# 파이썬을 이용한 웹 자동화
# 출처 : https://github.com/minyong-shin/Bloging/blob/master/MY%20-%20%EC%9C%A0%ED%8A%9C%EB%B8%8C%20info%20%EB%B0%8F%20%EB%8C%93%EA%B8%80%20%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb
# 크롤링에 필요한 모듈 임포트
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점
import time # 사이트를 불러올 때, 작업 지연 시간을 지정해주기 위한 패키지(사이트가 늦게 켜지면 에러가 발생하기 때문)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import urllib.request 
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt


# url을 불러오기 위한 사전작업 실행
# selenium 은 간단히 말해서 컴퓨터 = 나 
# 로딩이 끝날 때까지 기다려?
delay = 3
browser = Chrome('C:\chromedriver\chromedriver.exe')
browser.implicitly_wait(delay)

# 유튜브 url로 접속
start_url  = 'https://www.youtube.com'
browser.get(start_url)  
browser.maximize_window()

# 검색창 찾기
# //*[@id="search"] -> 검색창 input태그 xpath 개발자 도구에서 복사한값
browser.find_elements_by_xpath('//*[@id="search"]')[0].click() # 검색창 영역 클릭
browser.find_elements_by_xpath('//*[@id="search"]')[0].send_keys('양띵 사생활') # 검색창 영역에 원하는 유튜버 입력
browser.find_elements_by_xpath('//*[@id="search"]')[0].send_keys(Keys.RETURN) # 엔터

# 이동한 화면에서 유튜버를 클릭함 
#browser.find_elements_by_xpath('//*[@class="yt-simple-endpoint style-scope ytd-channel-render"]/div[2]/h3/span')[0].click()
browser.find_elements_by_xpath('//*[@id="main-link"]')[0].click()

# 유튜버에 들어간 뒤 카테고리가 보이는데 여기서 동영상 카테고리 클릭
browser.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div').click()

#스크롤하기 위해 소스 추출
body = browser.find_element_by_tag_name('body')

# 유튜브는 스크롤을 내리지 않으면 모든 영상들이 page source에 보이지 않기 때문에 스크롤을 
# 내리는 작업을 선행해줍니다
# (모든 스크롤을 내리지 않고 10회정도 한정해서 스크롤)
num_of_pagedowns = 1
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    num_of_pagedowns -= 1

# 위의 과정을 거치면 거의 모든 영상정보에 대해서 source가 보이는 것을 확인할 수 있음. 
# selenium 은 오래 걸리기 때문에 selenium 으로 필요한 부분까지만 진행하고 나머지는 beautiful, request로 작업
# 하는 것이 시간과 노력이 절감됨
# 내가 작업하고 있는 환경이 page Source를 추 출하는 코드 

html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
html

<html lang="ko-KR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif; "><head><script data-original-src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/annotations_module.js" src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/annotations_module.js"></script><script data-original-src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/endscreen.js" src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/endscreen.js"></script><script data-original-src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/captions.js" src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/captions.js"></script><script data-original-src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/miniplayer.js" src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/miniplayer.js"></script><script data-original-src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/remote.js" src="/yts/jsbin/player_ias-vflQm4drh/ko_KR/remote.js"></script><meta content="AhbmRDASY7NuOZD9cFMgQihZ+mQpCwa8WTGdTx82vSar9ddBQbziBfZXZg+ScofvEZDdHQNCEwz4yM7HjBS9RgkAAABneyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJXZW

In [21]:
# 영상들을 스크롤 한 뒤 page source로 변환하여 url을 수집
#video_list0 = html.find('div', {'id':'items'})
video_list0 = html.find('ytd-grid-renderer', {'class':'style-scope ytd-item-section-renderer'})
#video_lilst0 = video_list0.find('div',{'id':'items'})
video_list2 = video_list0.find_all('ytd-grid-video-renderer', {'class':'style-scope ytd-grid-renderer'})
video_list2

[<ytd-grid-video-renderer class="style-scope ytd-grid-renderer" lockup=""><div class="style-scope ytd-grid-video-renderer" id="dismissable"><ytd-thumbnail class="style-scope ytd-grid-video-renderer" use-hovered-property="" width="210">
 <a aria-hidden="true" class="yt-simple-endpoint inline-block style-scope ytd-thumbnail" href="/watch?v=dJECHAOllF0" id="thumbnail" rel="null" tabindex="-1">
 <yt-img-shadow class="style-scope ytd-thumbnail no-transition" loaded="" style="background-color: transparent;"><img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/dJECHAOllF0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&amp;rs=AOn4CLAVEPbGfdq7tYJJGXUn_ijb2hbf5A" width="210"/></yt-img-shadow>
 <div class="style-scope ytd-thumbnail" id="overlays"><ytd-thumbnail-overlay-time-status-renderer class="style-scope ytd-thumbnail" overlay-style="DEFAULT"><yt-icon class="style-scope ytd-thumbnail-overlay-time-status-renderer" disable-upgrade="" hidden=""></yt-

In [22]:
# 영상별로 url을 뽑기 위해 page source를 수집한 뒤 데이터를 추출
base_url = 'http://www.youtube.com' #영상별 url 이 유튜브 기본 url을 포함 하지 않기 때문에 꼭 확인해줘야함
isabe_url = []
for i in range(len(video_list2)):
    url = base_url+video_list2[i].find('a', {'id':'thumbnail'})['href']
    isabe_url.append(url)


In [23]:
isabe = pd.DataFrame({'name':[],
                     'thumbnail':[],
                     'view':[],
                     'previous_time':[],
                     'video_url':[],
                     'start_date':[],
                     'comment':[],
                     'likes_num':[],
                     'unlikes_num':[]})
#base_url = 'http://www.youtube.com'
delay = 3
browser.implicitly_wait(delay)
browser.maximize_window()

for i in range(0,10):
    name = video_list2[i].find('a',{'id':'video-title'}).text
    
    thum = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src']
    
    url =  isabe_url[i]
        
    
    meta0 =video_list2[i].find('div',{'id':'metadata-line'})
    
    view = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[0].text.split()[1]
    #조회수는 숫자만 뽑을 수 있고 모든 데이터는 원하는 형식으로 뽑을 수 있음
    previous = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[1].text
    
    start_url = isabe_url[i]
    browser.get(start_url) # 영상 하나하나 들어감
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(1.5)#브라우저 로딩시간기다려야함
    
    num_of_pagedowns = 4 #page_down은 2로 놔도 괜찮을 듯 최대화 했으므로
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5) #스크롤 속도를 지정할 수 있는 것인데  1.5초가 가장 적당할 듯 1초면 너무 빠름
        #현재는 스크롤을 7번만 하도록 설정되어 있으나 이후에 댓글을 크롤링하기 위해서는 스크롤을 댓글이 안보
        #일때까지 내리면 될텐데 시간이 엄청 오래 걸릴 것임 - 따라서 적당한 수준의 댓글을 크롤링하는것이 좋음
        num_of_pagedowns -= 1
    #여기에 time.sleep를 집어 넣는 방법을 고안
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    
  #  start_date = html.find('span',{'class':'date style-scope ytd-video-secondary-info-renderer'}).text
    start_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text
    comment = html.find('h2',{'id':'count'}).find('yt-formatted-string').text
    
    #좋아요수
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'

    #싫어요수
    unlikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    insert_data = pd.DataFrame({'name':[name],
                     'thumbnail':[thum],
                     'view':[view],
                     'previous_time':[previous],
                     'video_url':[url],
                     'start_date':[start_date],
                     'comment':[comment],
                     'likes_num':[likes_num],
                     'unlikes_num':[unlikes_num]})
    
    isabe = isabe.append(insert_data)
isabe.index = range(len(isabe))
isabe

,name,thumbnail,view,previous_time,video_url,start_date,comment,likes_num,unlikes_num
0,누텔라 싱크로율 100%! 택키글루의 매력을 뿜어내는 뭉슬라임 맛있는 리뷰,https://i.ytimg.com/vi/dJECHAOllF0/hqdefault.j...,5.5만회,3일 전,http://www.youtube.com/watch?v=dJECHAOllF0,2020. 3. 7.,댓글 123개,1.1천개,38개
1,하아... 이거 먹어버릴까요? 맛있어보이는 르몽드슬라임 리뷰!,https://i.ytimg.com/vi/7c57khw43M0/hqdefault.j...,5.5만회,6일 전,http://www.youtube.com/watch?v=7c57khw43M0,2020. 3. 4.,댓글 103개,1.1천개,24개
2,*역대급*헐.. 너무 좋아 외국슬라임 분위기 나는 이 슬라임 뭐야!!,https://i.ytimg.com/vi/TTGevW2hrH4/hqdefault.j...,9만회,1주 전,http://www.youtube.com/watch?v=TTGevW2hrH4,2020. 3. 1.,댓글 269개,1.5천개,44개
3,*슈팅스타*랑 완전 똑같아! 폼볼특집 신상연구소,https://i.ytimg.com/vi/mjomrJWKAxI/hqdefault.j...,7.8만회,2주 전,http://www.youtube.com/watch?v=mjomrJWKAxI,2020. 2. 22.,댓글 323개,1.3천개,39개
4,초보들을 위한 패키지?? 띵이슬라임 초보박스 리뷰!!,https://i.ytimg.com/vi/uIvye-9TfCU/hqdefault.j...,12만회,2주 전,http://www.youtube.com/watch?v=uIvye-9TfCU,2020. 2. 20.,댓글 433개,2.2천개,51개
5,마지막마켓...ㅠㅠ 폼볼맛집 02슬라임리뷰!,https://i.ytimg.com/vi/Wmk90Yvt_dg/hqdefault.j...,7.9만회,3주 전,http://www.youtube.com/watch?v=Wmk90Yvt_dg,2020. 2. 16.,댓글 194개,1.3천개,39개
6,🥛몰티져스가 아몬드우유에 퐁당? 신상5종 띵이슬라임!,https://i.ytimg.com/vi/zQqNkuglHvs/hqdefault.j...,11만회,1개월 전,http://www.youtube.com/watch?v=zQqNkuglHvs,2020. 2. 8.,댓글 320개,1.7천개,52개
7,도와주세요 맘에 드는게 너무 많아요ㅠㅠ,https://i.ytimg.com/vi/giUvTj8YXr0/hqdefault.j...,11만회,1개월 전,http://www.youtube.com/watch?v=giUvTj8YXr0,2020. 2. 6.,댓글 464개,1.9천개,49개
8,감옥세트? 독특한 컨셉의 디자인 맛집 두유슬라임 !,https://i.ytimg.com/vi/xBYH4HOUkOM/hqdefault.j...,7.3만회,1개월 전,http://www.youtube.com/watch?v=xBYH4HOUkOM,2020. 2. 4.,댓글 171개,1.3천개,33개
9,*인기컨텐츠* 띵이슬라임 30개 대용량 리뷰!,https://i.ytimg.com/vi/CjsyUnWsOLw/hqdefault.j...,30만회,1개월 전,http://www.youtube.com/watch?v=CjsyUnWsOLw,2020. 1. 23.,댓글 798개,4.2천개,147개
